In [ ]:
import numpy as np 
import xarray as xr
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt 
from matplotlib.animation import FuncAnimation

plt.rcParams['animation.html'] = 'jshtml'


def calc_perc_temp(src): 
    # number of valid nodes above minimum ice-thickness
    # NN = src.height.where(src.height < 10).count(dim=('nMesh_node'))#.count(dim=('coord_1', 'coord_2'))
    NN = src.height.where(src.height > 10).count(dim=('coord_1', 'coord_2'))

    # mask for determining where nodes are temperate 
    mask = (src['enthalpy_h'] > src['phase change enthalpy']) & (src.height > 10)
    # number of nodes where ethalpy exceeds phase chnage enthalpy
    NT = src.enthalpy_h.where(mask).count(dim=('coord_1', 'coord_2'))
    return NT/NN * 100



def load_runs(fp): 
    arrays = []

    for dt in [1.0, 0.5, 0.25, 0.1]: 
        # open both the heat source datasets for the given timesteps
        src = xr.open_mfdataset(fp.format(dt))
        # a little basic prepocessing 
        src = src.isel(Delta_MB=0, T_ma=0)
        src = src.rename({"t":"time"})

        if 1/src.dt != 1: 
            src = src.coarsen(time=10,boundary='trim').mean()
            src['time'] = np.floor(src.time)

        src['time'] = src.time * 365
        src.time.attrs = {"units": "days since 0001-01-01"}    
        # Decode the timestamp 
        src = xr.decode_cf(src, use_cftime=True)
        # Take the annual averages 
        src = src.groupby('time.year').mean()
    
        arrays.append(src)
    
    return arrays

In [ ]:
fp = 'glc1-a/nc/glc1-a_dx_50_NT_*_dt_{}_volumetric_prog_gridded.nc'
season_vol = load_runs(fp) 

In [ ]:
test = season_vol[0].height.isel(coord_2 = -1).integrate('coord_1') /\
       season_vol[0].height.isel(coord_2 = -1, year= 0).integrate('coord_1') 

In [ ]:
test.plot(x="year")

In [ ]:
fig, ax = plt.subplots()

for i, dt  in enumerate([1.0, 0.5, 0.25, 0.1]): 
        
    (season_vol[i].enthalpy_h.mean(('coord_2', 'coord_1'))/1e3).plot(x="year", marker='x', 
                                                                     ax=ax, label=f'dt={dt}')

plt.legend();
ax.set_title('Seasonality, Volumetric Heat Souce [J m$^{-3}$ a$^{-1}$]')
ax.set_ylabel('Mean Enthalpy [kJ kg$^{-1}$]');

ax.set_ylim(115, 130)


fig.savefig('figs/mean_Enth_Season_Vol.png', 
            dpi=300, bbox_inches='tight', facecolor='w')

In [ ]:
fp = 'glc1-a/nc/noSeasonality/glc1-a_dx_50_NT_*_dt_{}_volumetric_prog_gridded.nc'

noseason_vol = load_runs(fp) 

In [ ]:
fig, ax = plt.subplots()

for i, dt  in enumerate([1.0, 0.5, 0.25, 0.1]): 
    
    (noseason_vol[i].enthalpy_h.mean(('coord_2', 'coord_1'))/1e3).plot(x="year", marker='x', ax=ax, label=f'dt={dt}')

plt.legend()


ax.set_title('No Seasonality, Volumetric Heat Souce [J m$^{-3}$ a$^{-1}$]')
ax.set_ylabel('Mean Enthalpy [kJ kg$^{-1}$]');

ax.set_ylim(115, 130)


fig.savefig('figs/mean_Enth_noSeason_Vol.png', 
            dpi=300, bbox_inches='tight', facecolor='w')

In [ ]:
fp = 'glc1-a/nc/noSeasonality/glc1-a_dx_50_NT_*_dt_{}_mass_prog_gridded.nc'

noseason_mass = load_runs(fp) 

In [ ]:
fig, ax = plt.subplots()

for i, dt  in enumerate([1.0, 0.5, 0.25, 0.1]): 
    
    (noseason_mass[i].enthalpy_h.mean(('coord_2', 'coord_1'))/1e3).plot(x="year", marker='x', ax=ax, label=f'dt={dt}')

plt.legend();

ax.set_title('No Seasonality, Mass Heat Souce [J kg$^{-1}$ a$^{-1}$]')
ax.set_ylabel('Mean Enthalpy [kJ kg$^{-1}$]');

ax.set_ylim(115, 130)

fig.savefig('figs/mean_Enth_noSeason_Mass.png', 
            dpi=300, bbox_inches='tight', facecolor='w')

In [ ]:
# test = xr.open_mfdataset('glc1-a/nc/glc1-a_dx_50_NT_*_dt_*_*_prog_gridded.nc')

with xr.open_dataset('glc1-a/nc/glc1-a_dx_50_NT_500_dt_1.0_mass_prog_gridded.nc') as src: 
    src = src.sel(dt=1.0, heat_source='mass').isel(Delta_MB=0, T_ma=0)
    
    
    src['vol'] =src.height.isel(coord_2=-1).integrate("coord_1") /\
                src.height.isel(coord_2=-1,t=0).integrate("coord_1")
    
    src = src.rename({"t":"time"})
    src['time'] = np.floor(src.time * 365)
    src.time.attrs = {"units": "days since 2000-01-01"}    
    
    src = xr.decode_cf(src, use_cftime=True)
    
    
#     src.time.groupby('time.year').count(...).year

In [ ]:
test = xr.open_mfdataset('glc1-a/nc/glc1-a_dx_50_NT_*_dt_*_volumetric_prog_gridded.nc')

# Process time 
test = test.rename({"t":"time"})
# test['time'] = np.floor(test.time * 365)
# test.time.attrs = {"units": "days since 2000-01-01"}    

# test = xr.decode_cf(test, use_cftime=True)
    
    
# Calculate the relative volume
test['vol'] = test.height.isel(coord_2=-1).integrate("coord_1") /\
              test.height.isel(coord_2=-1, time=0).integrate("coord_1")
    
# test['Perc_temp'] = calc_perc_temp(test.isel(Delta_MB=0, T_ma=0))
test = test.isel(Delta_MB=0, T_ma=0)

In [ ]:
fig, ax = plt.subplots()

(test.enthalpy_h.mean(('coord_2', 'coord_1'))/1e3).sel(heat_source='volumetric').plot(x="time", hue='dt', ax=ax);

ax.set_ylim(112.5,130);

In [ ]:
test = xr.open_mfdataset('glc1-a/nc/noSeasonality/glc1-a_dx_50_NT_*_dt_0.25_*_prog_gridded.nc')
test = test.isel(Delta_MB=0, T_ma=0, dt=0)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(10,3), sharex=True, sharey=True, 
                       constrained_layout=True)

norm = plt.Normalize(-10, 0.0)

ax[0].set_xlim(0, 5)
ax[0].set_ylim(1950, 3000)


# def init():
#     im = ax[i].pcolormesh(test.X[:,::-1]/1e3, 
#                      test.Z.sel(heat_source=heat_source).isel(t=0),
#                      test.temperature.sel(heat_source=heat_source).isel(t=0),
#                      shading='gouraud', 
#                      norm=norm)
#     im.set_array([])
#     return im

for i, heat_source in enumerate(test.heat_source): 
    im = ax[i].pcolormesh(test.X[:,::-1]/1e3, 
                     test.sel(heat_source=heat_source).Z.isel(t=0),
                     test.temperature.sel(heat_source=heat_source).isel(t=0),
                     shading='gouraud', 
                     norm=norm)

    # Fill between bed and bottom of plot
    ax[i].fill_between(test.X[0,::-1]/1e3,
                    test.sel(heat_source=heat_source).isel(t=0,coord_2=-1).zbed,
                    y2=np.minimum(test.sel(heat_source=heat_source).isel(t=0,coord_2=-1).zbed.min(), 0.0),
                    color='gray', alpha=0.5)

# Add time annotation
label = ax[0].text(0.9, 0.9, r"$t=0$",
                transform=ax[0].transAxes,
                ha='center', va='center')

# Set up the colorbar
cbar = fig.colorbar(im, ax=ax)
cbar.set_label('Homologous Temperature [$^\circ$ C]', rotation=270, labelpad=20)
        
def update(t):
    for i, heat_source in enumerate(test.heat_source): 
        
        # Wipe the axis
        ax[i].clear()
        im = ax[i].pcolormesh(test.X[:,::-1]/1e3, 
                         test.sel(heat_source=heat_source).Z.isel(t=t),
                         test.temperature.sel(heat_source=heat_source).isel(t=t),
                         shading='gouraud', 
                         norm=norm)


        # Fill between bed and bottom of plot
        ax[i].fill_between(test.X[0,::-1]/1e3,
                        test.sel(heat_source=heat_source).isel(t=0,coord_2=-1).zbed,
                        y2=np.minimum(test.sel(heat_source=heat_source).isel(t=0,coord_2=-1).zbed.min(), 0.0),
                        color='gray', alpha=0.5)
        # Update time annotation
        # label.set_text("$t={{{:6.1f}}}$".format(src.t[i].values))

        ax[i].set_title(str(heat_source.values))
        
    # Add time annotation
    label = ax[0].text(0.9, 0.9, "t={}".format(test.t[t].values),
                    transform=ax[0].transAxes,
                    ha='center', va='center')

    
    ax[0].set_xlim(0, 5)
    ax[0].set_ylim(1950, 3000)

ani = FuncAnimation(fig, update, frames=np.arange(2, 500, 10), interval=100)
plt.close()

In [ ]:
ani.save('figs/noSeasonality_dt_0.25.mp4', dpi=400)

In [ ]:
test = xr.open_mfdataset('glc1-a/nc/glc1-a_dx_50_NT_*_dt_*_*_prog_gridded.nc', 
                        compat="identical")

In [ ]:
test.heat_source

In [ ]:
src = test.isel(t=-1, Delta_MB=0, T_ma=0, dt=0)


fig, ax = plt.subplots(2,1) 

for i, heat_source in enumerate(src.heat_source): 
    
    ax[i].pcolormesh(src.X[:,::-1], 
                     src.Z.sel(heat_source=heat_source),
                     src.temperature.sel(heat_source=heat_source))
                     


In [ ]:
src = test.sel(dt=0.1, heat_source='volumetric').isel(Delta_MB=0, T_ma=0, t=-3)


plt.pcolormesh(src.X[:,::-1], 
               src.Z,
               src.temperature)

In [ ]:
test.sel(dt=0.1, heat_source='volumetric').isel(Delta_MB=0, T_ma=0, t=-100)